<a href="https://colab.research.google.com/github/NelsenTimoty/com_bio/blob/main/ComBio_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install biopython

In [ ]:
%pip install catboost

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
import time
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import cm
from tqdm import tqdm
from Bio.Seq import Seq
from sklearn.model_selection import KFold,GroupKFold,GridSearchCV,StratifiedKFold
from sklearn.metrics import confusion_matrix,balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer,RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.model_selection import learning_curve,ShuffleSplit
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import os,warnings;warnings.filterwarnings("ignore")

In [ ]:
class get_unitigs:

    def __init__(self,verbose=True):
        self.df = pd.read_csv('metadata.csv', index_col=0) # metadata
        self.meta_names = self.df.columns
        self.target_name = None
        self.verbose = verbose

    # Get Unitig Feature matrix & Target Vector
    def get_case(self,phenotype=None):

        self.target_name = phenotype
        _metadata = self.df
        if(self.verbose):
            print(f'Target Antibiotic: {self.target_name}')
            print(f'Metadata df: {_metadata.shape}')

        # remove those that don't contain target values
        _metadata = _metadata.dropna(subset=[phenotype])
        self.metadata = _metadata.copy()

        if(self.verbose):
            print(f'Metadata df after na() removal {_metadata.shape}')
        _metadata = _metadata[phenotype] # choose target variable

        prefix = ''
        suffix = '_gwas_filtered_unitigs.Rtab'

        if(self.verbose):
            print('\nCombining Metadata & Unitigs')

        # unitig feature matrix for phenotype
        tdf = pd.read_csv(prefix + phenotype + suffix, sep=" ",
                          index_col=0, low_memory=False)
        # align column data w/ metadata df (pattern_id = sample_idd)
        tdf = tdf.T
        # keep only common rows, ie. that have resistence measure]
        tdf = tdf[tdf.index.isin(_metadata.index)]

        train = tdf
        target = _metadata[_metadata.index.isin(tdf.index)]

        self.X = pd.concat([train,target],axis=1)
        if(self.verbose):
            print(f'Unitig Matrix (+target): {self.X.shape}')

In [ ]:
case_cip = get_unitigs()
display(case_cip.df.T.iloc[:,:7])

Sample_ID,ERR1549286,ERR1549290,ERR1549291,ERR1549287,ERR1549288,ERR1549299,ERR1549292
Year,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0
Country,UK,UK,UK,UK,UK,UK,UK
Continent,Europe,Europe,Europe,Europe,Europe,Europe,Europe
Beta.lactamase,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Azithromycin,>256,>256,>256,>256,>256,>256,>256
Ciprofloxacin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ceftriaxone,0.016,0.004,0.006,0.006,0.008,0.012,0.023
Cefixime,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tetracycline,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Penicillin,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
case_cip.get_case(phenotype='cip_sr')
# case_cip.X.iloc[:,:2].head()

Target Antibiotic: cip_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3088, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3088, 8874)


In [ ]:
case_cip.get_case(phenotype='cfx_sr')
# case_cip.X.iloc[:,:2].head()

Target Antibiotic: cfx_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3401, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3401, 385)


In [ ]:
case_cip.get_case(phenotype='azm_sr')
# case_cip.X.iloc[:,:2].head()

Target Antibiotic: azm_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3478, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3478, 516)


The target vector, is also stored in X, and represents the resistance property of the sample
The Sample_ID is either 1.0 (resistant) or 0.0 (non-resistant) to a particular antibiotic

In [ ]:
target = case_cip.X[case_cip.target_name]
print(target[:5])

ERR1549286    1.0
ERR1549290    1.0
ERR1549291    1.0
ERR1549287    1.0
ERR1549288    1.0
Name: azm_sr, dtype: float64


TARGET DISTRIBUTIONS & UNITIG COUNT

In [ ]:
case = get_unitigs()
case.get_case('cip_sr')
print(case.X[case.target_name].value_counts())

Target Antibiotic: cip_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3088, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3088, 8874)
cip_sr
0.0    1660
1.0    1428
Name: count, dtype: int64


In [ ]:
case = get_unitigs()
case.get_case('azm_sr')
print(case.X[case.target_name].value_counts())

Target Antibiotic: azm_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3478, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3478, 516)
azm_sr
0.0    3031
1.0     447
Name: count, dtype: int64


In [ ]:
case = get_unitigs()
case.get_case('cfx_sr')
print(case.X[case.target_name].value_counts())

Target Antibiotic: cfx_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3401, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3401, 385)
cfx_sr
0.0    3396
1.0       5
Name: count, dtype: int64


the concentration of antibiotic bacteria can tolerate before it impairs their growth.

In [ ]:
def get_mic():

    lst_cases = ['azm_mic','cip_mic','cfx_mic']
    rtabs = ['azm_sr','cip_sr','cfx_sr']
    lst_temp = []

    ii=-1
    for case in lst_cases:

        ii+=1
        case_id = get_unitigs(verbose=False)
        case_id.get_case(rtabs[ii])

        X_all = pd.concat([case_id.X,case_id.metadata],axis=1)

        new_df = X_all[case].value_counts().rename_axis(case).reset_index(name='counts')
        new_df = new_df.rename(columns={new_df.columns[0]: 'mic'})
        new_df['case'] = case
        lst_temp.append(new_df)

    X_counts = pd.concat([lst_temp[0],lst_temp[1],lst_temp[2]],axis=0)
    X_counts.sort_values(by='mic',inplace=True,ascending=True)
    X_counts['mic'] = X_counts['mic'].astype(str)

    fig = px.bar(X_counts, x='mic',y='counts',color='case')
    fig.update_layout(template='plotly_white',height=300)
    fig.show()

In [ ]:
get_mic()

CHECK UNIQUE VALUES OF ALL UNITIGS

In [ ]:
column_values = case.X[case.target_name].values.ravel()
print(pd.unique(column_values))

[0. 1.]


In [ ]:
case_unitigs = case.X.columns.tolist()

ii=-1
for i in case_unitigs:
    if(',' in i):
        ii+=1;print(f'{ii} | {i}')

0 | TTCAACTCTTCATAAGCCAAAGTCTGAATCCTCTGATC,AACAGCTCCTTAACCTGCGTGCGGTACAGGCGTTCGGCGCAGGC
1 | CTGTTTGTCTTTTTGGCGGGCGGTATGCTGA,CTGCATCCGGGGACATCCGTCGTAATCACCGCCCTGC
2 | CGCACGACGGCGTTTTGCTGCCGCTCAGCTTTGAGAAGCAGG,AAGGTTCGATCAGGAAGCCGTTGACCTTGTCGGC
3 | AAAACATCGGGGCGGTCATACCCGAATTGCGCCCCAAAGAA,TTCTCGACTGGGCAATTTTCCAGTGTCAATCCTTTGGTCTTGGTTTCCAACAGGTCTAGG
4 | GATAAGGGCAAACATCTGCGCGGCTTTGCGTCCGGAATAATAATCGCGC,CATGAAGCCGTTTACCATTGCCAAAGCATTGGATTC
5 | GCCGGTCAGCTTTGAAAAACAAGCGGTTGCGCCTAAAGGCAAGCGCGTCATCAAAGCCTCT,AATAGAGTCTGAGTTTGTTTATGTTCGGCACGGCGTTC
6 | CAATACGCCCGCCTACGATCCCAACAGACCCGGC,GAGCGCGCCGACGGAGTCCCTGTTTGCCGTACCTAAAGAGATGAAGGAAATGCCGTCTGCC,CTCGATATTTTGAAGGCGTAAACAAGGTGGTCGGCTTTT,CAATATTGTGGACAGTTTGGATTCTCCGCGCAATAAAGCCCCGCAAAACG,ATTGGATGCGGGCAAAACCGATTTGAACGAACGGCTGAATACGCAGCCT,GCGGTCATACCCGAATTGCGCCCCAAAGAAACCAGC
7 | ACTCCATGAAGAACTCAGGGAAGAACACGCCCGCCGCCCTGTCGGAA,TGCCGAACATAAACAAACTCAGACTCTATTCGATAC
8 | CTACGATCCCAACAGACCCGGCCGGGCAGACAGCGA,AATGCCGTCTGCCGCCCAATTGGAACGCCTGTCCGAGCTT,GGC

WORKING WITH SEQUENCE

In [ ]:
class SQ:
    def __init__(self, sequence, seq_type):
        self.sequence = sequence
        self.seq_type = seq_type

lst_SQ = []
for unitig in case_unitigs:
    lst_SQ.append(SQ(unitig,'DNA'))

print(type(lst_SQ[4]))

lst_bSQ = []
for unitig in case_unitigs:
    lst_bSQ.append(Seq(unitig))

print(type(lst_bSQ[4]))

<class '__main__.SQ'>
<class 'Bio.Seq.Seq'>


CREATING RESISTANCE MODELS

In [ ]:
models = []
models.append(
    ('LDA', LinearDiscriminantAnalysis()))

# models use in gscv
cv_lda = {'solver':['svd', 'lsqr', 'eigen']}

models = []
models.append(
    ('LDA', LinearDiscriminantAnalysis(),cv_lda))

EVALUATE THE MODEL

In [ ]:
class class_eval():

    def __init__(self,data=None,models=None,nfold = 4,
                 gsfold=3, verbose=True,
                 shuffle_id=False,random_state=22):

        self.shuffle = shuffle_id

        if(self.shuffle is False):
            self.rs = None
        else:
            self.rs = random_state # random state

        if(data is not None):
            X_all = data.X
            self.y = data.X[data.target_name].copy()
            self.X = data.X.drop([data.target_name],axis=1).copy()
        else:
            print('input case data')

        self.models = models # Tuple /w (name,model)
        self.nfold = nfold       # number of cross validation folds
        self.gsfold = gsfold    # number of grid search folds in every fold
        self.target_name = data.target_name  # target name

    ''' Grid-Search Standard Kfold Cross Validation '''
    # For each fold, a grid search cv is applied to find best params

    def gscv(self,type_id='kfold'):

        self.store_models = {} # Store models of Kfolds
        self.dic_tr = {}
        self.dic_te = {}

        for model in self.models:

            if(type_id is 'skfold'):
                kfold = StratifiedKFold(n_splits=self.nfold,shuffle=self.shuffle,random_state=self.rs)
            elif(type_id is 'kfold'):
                kfold = KFold(n_splits=self.nfold,shuffle=self.shuffle,random_state=self.rs)

            lst_temp = []; lst_temp2 = []; kfold_id = -1
            for train_index, test_index in kfold.split(self.X,self.y):

                kfold_id+=1;print(f'\nkfold {kfold_id}')
                # split data into train/test sets
                X_train = self.X.iloc[train_index]
                y_train = self.y[train_index]
                X_test = self.X.iloc[test_index]
                y_test = self.y[test_index]

                # perform grid search to identify best hyper-parameters
                gs_clf = GridSearchCV(model[1],param_grid=model[2],
                                      cv=self.gsfold, n_jobs=-1,
                                      scoring='balanced_accuracy')

                # Train using the best model
                gs_clf.fit(X_train, y_train)
                best_model = gs_clf.best_estimator_ # best cv model
                self.store_models[f'GS_{model[0]}_{kfold_id}'] = best_model

                # Predict using best model
                ym_tr = gs_clf.predict(X_train)
                ym_te = gs_clf.predict(X_test)
                score_tr = balanced_accuracy_score(y_train,ym_tr)
                score_te = balanced_accuracy_score(y_test,ym_te)
                lst_temp.append(score_tr); lst_temp2.append(score_te)

                print(f'Train Score: {round(score_tr,4)} - Test Score: {round(score_te,4)}')
                print('Best hyperparameters for this fold')
                print(gs_clf.best_params_)
                print(f"Test : Confusion matrix Fold {kfold_id}")
                print(confusion_matrix(y_test, ym_te))

            self.dic_tr[model[0]] = lst_temp
            self.dic_te[model[0]] = lst_temp2
            test_mean = round(sum(lst_temp2)/len(lst_temp2),4)
            print(f'\n {model[0]} - Test Mean Score: {test_mean}')

    '''K-Fold Cross Validation'''
    # w/ type_id option; statified used to make sure classes are balanced in folds

    def cv(self,type_id='kfold'):

        print(f'type_id set to: {type_id}')
        self.store_models = {} # Store models of Kfolds
        self.dic_tr = {}
        self.dic_te = {}

        # Cycle though all tuple model settings
        for model in self.models:

            t0 = time.time()
            if(type_id is 'skfold'):
                kfold = StratifiedKFold(n_splits=self.nfold,
                                        shuffle=self.shuffle,
                                        random_state=self.rs)
            elif(type_id is 'kfold'):
                kfold = KFold(n_splits=self.nfold,
                              shuffle=self.shuffle,
                              random_state=self.rs)

            lst_temp = []; lst_temp2 = []; kfold_id = -1
            for train_index, test_index in kfold.split(self.X,self.y):

                kfold_id+=1
                # split data into train/test sets
                X_train = self.X.iloc[train_index]
                y_train = self.y[train_index]
                X_test = self.X.iloc[test_index]
                y_test = self.y[test_index]

                # Fit, Stopre & Predict Kfold models
                model[1].fit(X_train, y_train)
                self.store_models[f'{model[0]}_{kfold_id}'] = model[1]
                ym_tr = model[1].predict(X_train)
                ym_te = model[1].predict(X_test)

                score_tr = balanced_accuracy_score(y_train,ym_tr)
                score_te = balanced_accuracy_score(y_test,ym_te)
                lst_temp.append(score_tr); lst_temp2.append(score_te)

            self.dic_tr[model[0]] = lst_temp
            self.dic_te[model[0]] = lst_temp2
            test_mean = round(sum(lst_temp2)/len(lst_temp2),4)
            print(f'{model[0]} - Test Mean Score: {test_mean} - Total Time: {round(time.time() - t0,4)}')

    ''' unitig visual functions only '''

    def col_trans(self,sel_id=0):

        # unitig to abrev
        if(sel_id is 0):
            self.X_names = self.X.columns.tolist()
            temp_names = self.X_names.copy()

            unitigs = self.X_names.copy()
            del unitigs[-1]

            lst_abr = []
            ii=-1
            for unitig in range(0,len(unitigs)):
                ii+=1;lst_abr.append(f'u{ii}')
            lst_abr.append(self.target_name)
            self.X.columns = lst_abr
            self.dicabr = dict(zip(lst_abr,self.X_names))

        # abreb to unitig
        elif(sel_id is 1):
            self.X.columns = self.X_names

    ''' PLOT KFOLD RESULTS '''

    def fold_plot(self):

        df_tr = pd.DataFrame(self.dic_tr)
        df_te = pd.DataFrame(self.dic_te)

        # Make Plots
        fig,ax = plt.subplots(2,2,figsize=(14,6))
        sns.stripplot(data=df_tr, orient='h',linewidth=1,ax=ax[0,0])
        sns.heatmap(data=df_tr,annot=True,cbar=False,cmap="plasma_r",vmax=1,vmin=0.5,ax=ax[0,1])
        sns.stripplot(data=df_te, orient='h',linewidth=1,ax=ax[1,0])
        sns.heatmap(data=df_te,annot=True,cbar=False,cmap="plasma_r",vmax=1,vmin=0.5,fmt='.3g',ax=ax[1,1])
        ax[0,0].set_xlim([0.5,1.1]);ax[1,0].set_xlim([0.5,1.1])
        ax[0,0].set_title('train kfold')
        ax[0,1].set_title('train kfold heatmap')
        ax[1,0].set_title('test kfold')
        ax[1,1].set_title('test kfold heatmap')
        sns.despine(bottom=True, left=True)
        plt.tight_layout()

In [ ]:
models = []
n_est = 10

models.append(('LR', LogisticRegression()))
models.append(('SVM',SVC()))
models.append(('RF', RandomForestClassifier(n_estimators=n_est)))

RESULT

In [ ]:
case = get_unitigs()
case.get_case('cip_sr')

# Standard KFOLD evaluation
eval1 = class_eval(data=case, nfold = 4, models=models)
eval1.cv(type_id='kfold')

Target Antibiotic: cip_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3088, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3088, 8874)
type_id set to: kfold
LR - Test Mean Score: 0.9486 - Total Time: 17.4542
SVM - Test Mean Score: 0.9473 - Total Time: 99.7474
RF - Test Mean Score: 0.9434 - Total Time: 2.0095


In [ ]:
case = get_unitigs()
case.get_case('cfx_sr')

# Standard KFOLD evaluation
eval1 = class_eval(data=case,nfold = 4, models=models)
eval1.cv(type_id='kfold')

Target Antibiotic: cfx_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3401, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3401, 385)
type_id set to: kfold
LR - Test Mean Score: 0.625 - Total Time: 0.895
SVM - Test Mean Score: 0.625 - Total Time: 0.5197
RF - Test Mean Score: 0.625 - Total Time: 0.3219


In [ ]:
case = get_unitigs()
case.get_case('azm_sr')

# Standard KFOLD evaluation
eval1 = class_eval(data=case, nfold = 4, models=models)
eval1.cv(type_id='kfold')

Target Antibiotic: azm_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3478, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3478, 516)
type_id set to: kfold
LR - Test Mean Score: 0.8839 - Total Time: 2.0252
SVM - Test Mean Score: 0.8854 - Total Time: 5.0135
RF - Test Mean Score: 0.7906 - Total Time: 0.3506
